# Colab-image-classification

In [ ]:
!nvidia-smi

In [ ]:
#@title Install
!pip install kornia efficientnet_pytorch x_transformers vit-pytorch swin-transformer-pytorch adamp tensorboardX torchvision timm madgrad
!pip install git+https://github.com/styler00dollar/pytorch-lightning.git@fc86f4ca817d5ba1702a210a898ac2729c870112
!git clone https://github.com/styler00dollar/Colab-image-classification
%cd /content/Colab-image-classification

# Train

In [ ]:
%cd /content/Colab-image-classification
!python means_stds.py --train_dir '/content/data'

In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

In [ ]:
# delete ipynb_checkpoints file if needed, since dataloader does not a filetype check
!sudo find /content/ -iname "*ipynb_checkpoints*"
!sudo rm -rf /content/data/.ipynb_checkpoints

In [ ]:
#@title config.yaml
%%writefile /content/Colab-image-classification/config.yaml
print_training_epoch_end_metrics: False # RAM requirements for big datasets are high, disble it if you have big datasets. does not imply validation
use_amp: False
gpus: 0
use_swa: False
save_step_frequency: 5000

# calculate these with means_stds.py!
means: [0.7032, 0.6346, 0.6234]
std: [0.2520, 0.2507, 0.2417]

optimizer: AdamP # Adam | AdamP | SGDP | MADGRAD (pip install madgrad)
lr: 0.0001
batch_size: 2 # 62
aug: MuAugment # gridmix, cutmix, MuAugment, None
loss: normal # normal | CenterLoss
size: 256 # image size
precision: 32 # 32
max_epochs: 9999
progress_bar_refresh_rate: 20
default_root_dir: "/content/"

path:
  # pretrain
  pretrain:
  checkpoint_path: 

  training_path: '/content/data/' # input folder
  validation_path: '/content/data/' # validation folder
  test_path: '/content/data/' # validation folder / test.py
  log_path: "/content/logs/"
  checkpoint_save_path: "/content/" 


# which model architecture to train

# efficientnet-b0 up to efficientnet-b8 (pip install efficientnet_pytorch)
# mobilenetv3_small / mobilenetv3_large
# resnet50 / resnet101 / resnet152
# ViT / DeepViT
# RepVGG-A0, RepVGG-A1, RepVGG-A2, RepVGG-B0, RepVGG-B1, RepVGG-B1g2, RepVGG-B1g4, RepVGG-B2, RepVGG-B2g2, RepVGG-B2g4, RepVGG-B3, RepVGG-B3g2, RepVGG-B3g4
# squeezenet_1_0 / squeezenet_1_1
# vgg11, vgg13, vgg16, vgg19
# SwinTransformer

# for vgg and resnet
#pretrain: False

# timm
# pip install timm
# you can loop up models here: https://rwightman.github.io/pytorch-image-models/
# Example: "tf_efficientnetv2_s"
#model_train: timm
#model_choise: 'tf_efficientnetv2_b0'
#model_choise: "swin_tiny_patch4_window7_224"

#################

# other models with configurable parameters
#model_train: effV2
#conv: fft # fft | conv2d
#model_size: s # s | m | l | xl

# x-transformers
# pip install x-transformers
#model_train: x_transformers
#image_size: 512
#patch_size: 32
#dim: 512
#depth: 6
#heads: 8

# Warning: only 256px input (b110)
#model_train: mobilevit 
#model_size: xxs # xxs | xs | s

# because of too many parameters, a seperate config file named "hrt_config.yaml" is available
#model_train: hrt

# volo (2021) (224px)
#model_train: volo
#model_size: volo_d1 # volo_d1 up to volo_d5
#pretrain: False

# pvt_v2 (2021)
#model_train: pvt_v2
#model_size: pvt_v2_b0 # pvt_v2_b0 to pvt_v2_b5, pvt_v2_b2_li
#pretrain: False

# ConvMLP (2021)
#model_train: ConvMLP
#model_size: convmlp_s # convmlp_s | convmlp_m | convmlp_l
#pretrain: False

# FocalTransformer (2021)
#model_train: FocalTransformer

# MobileFormer (2021)
#model_train: mobile_former
#model_size: config_52 # config_52 | config_294 | config_508

# poolformer (2021)
model_train: poolformer
model_size: poolformer_s12 # poolformer_s12 | poolformer_s24 | poolformer_s36 | poolformer_m36 | poolformer_m48

#################

num_classes: 2 # Warning: Some do require amount classes + 1 as num_classes. 
diffaug_activate: False
policy: 'color,translation' # [color,translation,cutout]
num_workers: 12

################################################################
# inference
# args for sort.py / test.py (only supports efficientnet)
resize_method: PIL # PIL | OpenCV
model_path: '/content/model.pth'
path0: '/content/0'
path1: '/content/1'

In [ ]:
%cd /content/Colab-image-classification
!python train.py

----------------------------------------------------------------------

# TensorRT

Original Colab: [here](https://colab.research.google.com/drive/1oe_aflRfCwRehho_8QlD8YFKUQ_I5sA6#scrollTo=tgSrJdHic-Du)

Colab-torch2trt: [styler00dollar/Colab-torch2trt](https://github.com/styler00dollar/Colab-torch2trt/blob/main/Colab-torch2trt.ipynb)

onnx-tensorrt: [onnx/onnx-tensorrt](https://github.com/onnx/onnx-tensorrt)


TensorRT gives better performance. You need to get 2 files.
Currently, the cuda version inside Colab is 11.0, that's why you need to get:

```
nv-tensorrt-repo-ubuntu1804-cuda11.0-trt7.2.3.4-ga-20210226_1-1_amd64.deb

and
 
TensorRT-7.2.3.4.Ubuntu-18.04.x86_64-gnu.cuda-11.0.cudnn8.1.tar\TensorRT-7.2.3.4\python\tensorrt-7.2.3.4-cp37-none-linux_x86_64.whl (inside TensorRT-7.2.3.4.Ubuntu-18.04.x86_64-gnu.cuda-11.0.cudnn8.1.tar.gz)
```

You can download these files [here](https://developer.nvidia.com/nvidia-tensorrt-download). Warning: You need an account (which can be created for free).

If you want to use other versions, you need to adjust the install script.

Warning: Not everything works with tentorrt.
- Not working: efficientnet
- Working: Resnet
- Everything else is untested.

In [ ]:
!nvcc --version

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
#@title install
import os
os.environ["os1"]="ubuntu1804"
os.environ["tag"]= "cuda11.0-trt7.2.3.4-ga-20210226" #@param
os.environ["version"]= "7.2.3-1+cuda11.0" #@param
data_path = '/content/drive/MyDrive/tensorrt 11.0/' #@param
os.chdir(data_path)
!sudo dpkg -i nv-tensorrt-repo-${os1}-${tag}_1-1_amd64.deb
!sudo apt-key add /var/nv-tensorrt-repo-${tag}/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install libnvinfer7=${version} libnvonnxparsers7=${version} libnvparsers7=${version} libnvinfer-plugin7=${version} libnvinfer-dev=${version} libnvonnxparsers-dev=${version} libnvparsers-dev=${version} libnvinfer-plugin-dev=${version} python-libnvinfer=${version} python3-libnvinfer=${version}
!sudo apt-mark hold libnvinfer7 libnvonnxparsers7 libnvparsers7 libnvinfer-plugin7 libnvinfer-dev libnvonnxparsers-dev libnvparsers-dev libnvinfer-plugin-dev python-libnvinfer python3-libnvinfer
!sudo apt-get install tensorrt=${version}
!sudo apt-get install python3-libnvinfer-dev=${version}

**Restart colab (Runtime > Restart Runtime)**

In [ ]:
!pip install "/content/drive/MyDrive/tensorrt 11.0/tensorrt-7.2.3.4-cp37-none-linux_x86_64.whl"

In [ ]:
!python onnx_convert.py --model_train 'resnet50' --num_classes 3 --output_path "/content/output.onnx" \
  --model_path '/content/Colab-image-classification/Checkpoint_1_7_loss_0.162406_acc_0.666667_D.pth'

In [ ]:
#@title Install (pycuda, onnx, onnx-tensorrt)
%cd /content/
!pip install pycuda
!pip install onnx
#%cd /content/
!git clone https://github.com/onnx/onnx-tensorrt
%cd onnx-tensorrt
!python setup.py install

In [ ]:
#@title Example usage of tensorrt backend
import onnx
import onnx_tensorrt.backend as backend
import numpy as np
onnx_path = "/content/quant.onnx" #@param
model = onnx.load(onnx_path)
engine = backend.prepare(model, device='CUDA:0', fp16_mode=True)
input_data = np.random.random(size=(1, 3, 256, 256)).astype(np.float32)
output_data = engine.run(input_data)[0]
print(output_data)
print(output_data.shape)

In [ ]:
%cd /content/Colab-image-classification
# moving files with 2 classes example (onnx)
!python sort_onnx.py --data_input_path "/content/data/" --onnx_path '/content/test.onnx' \
  --path0 '/content/0/' --path1 '/content/1/' --num_classes 3 --height_min 256 --width_min 256

# ONNX Quantization
Maybe broken, use at your own risk.
- Not working: EfficientNet, Resnet
- Everything else is untested.

In [ ]:
!pip install onnx -U
!pip install onnxruntime-gpu

In [ ]:
import onnx
from onnxruntime.quantization import quantize_qat, QuantType

model_fp32 = '/content/output.onnx'
model_quant = '/content/quant.onnx'
quantized_model = quantize_qat(model_fp32, model_quant)

# Links


GridMixup: [IlyaDobrynin/GridMixup](https://github.com/IlyaDobrynin/GridMixup)

Centerloss: [KaiyangZhou/pytorch-center-loss](https://github.com/KaiyangZhou/pytorch-center-loss)

Cutmix: [hysts/pytorch_cutmix](https://github.com/hysts/pytorch_cutmix)

Diffaug: [mit-han-lab/data-efficient-gans](https://github.com/mit-han-lab/data-efficient-gans)

AdamP: [clovaai/AdamP](https://github.com/clovaai/AdamP)

EfficientNet repo: [lukemelas/EfficientNet-PyTorch](https://github.com/lukemelas/EfficientNet-PyTorch)

RepVGG: [DingXiaoH/RepVGG](https://github.com/DingXiaoH/RepVGG)

MobileNetV3: [kuan-wang/pytorch-mobilenet-v3](https://github.com/kuan-wang/pytorch-mobilenet-v3)

swin-transformer: [berniwal/swin-transformer-pytorch](https://github.com/berniwal/swin-transformer-pytorch)

Resnest: [zhanghang1989/ResNeSt](https://github.com/zhanghang1989/ResNeSt)

timm: [rwightman/pytorch-image-models](https://github.com/rwightman/pytorch-image-models)

madgrad: [facebookresearch/madgrad](https://github.com/facebookresearch/madgrad)

EfficientNetV2: [efficientnetv2.pytorch](https://github.com/d-li14/efficientnetv2.pytorch)

lama: [saic-mdal/lama](https://github.com/saic-mdal/lama)

x-transformers: [lucidrains/x-transformers](https://github.com/lucidrains/x-transformers)

mobilevit: [chinhsuanwu/mobilevit-pytorch](https://github.com/chinhsuanwu/mobilevit-pytorch)

hrt: [HRNet/HRFormer](https://github.com/HRNet/HRFormer)

MuarAugment: [adam-mehdi/MuarAugment](https://github.com/adam-mehdi/MuarAugment)

poolformer: [sail-sg/poolformer](https://github.com/sail-sg/poolformer)

Original repo: [bentrevett/pytorch-image-classification](https://github.com/bentrevett/pytorch-image-classification)

My fork: [styler00dollar/Colab-image-classification](https://github.com/styler00dollar/Colab-image-classification)